In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import serial
import time

# === 1. Load and Train Model ===
df = pd.read_csv(r"C:\Users\asus\Garbage-Sense\Trash_Fill_Data.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df = df.rename(columns={'fill_percentage_%': 'fill_percent', 'time_hrs': 'hours_elapsed'})
assert 'fill_percent' in df.columns and 'hours_elapsed' in df.columns, "Missing required columns"
df.dropna(subset=['fill_percent', 'hours_elapsed'], inplace=True)

# Train model: hours_elapsed (X) -> fill_percent (y)
X = df[['hours_elapsed']].values
y = df['fill_percent'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Model trained. Test MSE: {mse:.2f}")
print(f"Intercept: {model.intercept_:.2f}, Slope: {model.coef_[0]:.2f}")

# === 2. Prediction Function ===
def predict_time_to_full(current_percent, model):
    """Predict time to reach 100% with a max range of 9-11 hours."""
    if not 0 <= current_percent <= 100:
        return None
    
    slope = model.coef_[0]  # Fill % per hour
    intercept = model.intercept_
    
    # Inverse calculation: time = (fill_percent - intercept) / slope
    if slope <= 0:  # Invalid slope, use fallback
        print("Warning: Model slope is non-positive, using fallback estimate.")
        return None
    
    current_time = (current_percent - intercept) / slope
    time_at_full = (100 - intercept) / slope
    
    # Ensure realistic bounds
    if current_time < 0 or time_at_full <= current_time:
        print("Warning: Invalid time prediction, using fallback estimate.")
        return None
    
    remaining_time = time_at_full - current_time
    
    # Constrain total time to full between 9 and 11 hours
    min_time_to_full = 9.0
    max_time_to_full = 11.0
    
    # Scale remaining time based on remaining percentage
    remaining_percent = 100 - current_percent
    if remaining_time > max_time_to_full or remaining_time < 0:
        # Fallback: Linearly interpolate between 9-11 hours based on current fill
        max_possible_time = np.random.uniform(min_time_to_full, max_time_to_full)  # Random between 9-11
        remaining_time = (remaining_percent / 100) * max_possible_time
    
    # Ensure remaining time stays within bounds
    remaining_time = max(0, min(remaining_time, max_time_to_full))
    
    print(f"Debug: {current_percent}% -> Current Time: {current_time:.2f}h, Full Time: {time_at_full:.2f}h, Adjusted Remaining: {remaining_time:.2f}h")
    return remaining_time

# === 3. Serial Communication ===
def read_fill_percentage(serial_port):
    """Read fill percentage from Arduino."""
    try:
        line = serial_port.readline().decode('utf-8').strip()
        print(f"Raw data received: '{line}'")
        if line.startswith("PERCENT:"):
            return float(line.split("PERCENT:")[1])
        return None
    except (ValueError, UnicodeDecodeError, IndexError) as e:
        print(f"Error reading from serial: {e}")
        return None

# === 4. Main Loop ===
def main():
    arduino_port = 'COM5'  # Update to your Arduino's port
    baud_rate = 9600
    interval = 300  # 5 minutes in seconds

    # Connect to Arduino
    try:
        ser = serial.Serial(arduino_port, baud_rate, timeout=1)
        print(f"Connected to Arduino on {arduino_port}")
        time.sleep(2)  # Wait for connection to stabilize
    except serial.SerialException as e:
        print(f"Failed to connect to Arduino: {e}")
        return

    try:
        while True:
            fill_percent = read_fill_percentage(ser)
            if fill_percent is not None:
                predicted_time = predict_time_to_full(fill_percent, model)
                if predicted_time is not None:
                    print(f"Current Fill: {fill_percent:.0f}% | Time to reach 100%: {predicted_time:.2f} hours")
                else:
                    # Fallback: Linearly scale remaining time within 9-11 hours
                    remaining_percent = 100 - fill_percent
                    max_possible_time = np.random.uniform(9.0, 11.0)  # Random between 9-11
                    predicted_time = (remaining_percent / 100) * max_possible_time
                    print(f"Fallback: Current Fill: {fill_percent:.0f}% | Time to reach 100%: {predicted_time:.2f} hours")
            else:
                print("No valid percentage data received from Arduino.")
            print(f"Waiting {interval/60} minutes for next reading...")
            time.sleep(interval)
    except KeyboardInterrupt:
        print("\nStopped by user.")
    finally:
        ser.close()
        print("Serial connection closed.")

if __name__ == "__main__":
    main()

Model trained. Test MSE: 29.14
Intercept: 10.00, Slope: -0.42
Connected to Arduino on COM5
Raw data received: 'PERCENT:50'
Fallback: Current Fill: 50% | Time to reach 100%: 4.67 hours
Waiting 5.0 minutes for next reading...
